# collab connection


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



# Virtual env setup







In [ ]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [8]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="/content/drive/MyDrive/p2m/.env")

False

# openAI key

In [3]:
api_key = os.getenv("OPENROUTER_API_KEY")

In [9]:
%pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [10]:
from openai import OpenAI
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

# Text image splitter

## path,setup,extentions

In [1]:
%pip install marker-pdf[full]

In [4]:
import base64

# Liste des extensions considérées comme des images
IMAGE_EXTENSIONS = [".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"]
raw_data_folder = "/content/drive/MyDrive/p2m/datas"
marker_data_folder = "/content/drive/MyDrive/p2m/output_marker"


## image to txt , docx to md


## image description with llma4

In [4]:
!marker "$raw_data_folder" --output_dir "$marker_data_folder" --output_format 'markdown' --workers 1


2025-04-15 15:22:00.548114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744730520.787812    1803 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744730520.851262    1803 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 15:22:01.344527: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/20

In [ ]:
def image_to_text(image_path: str) -> str:
    # Convertir l'image en base64
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode("utf-8")

    # Envoyer la requête à l'API
    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "<YOUR_SITE_URL>",
            "X-Title": "<YOUR_SITE_NAME>",
        },
        model="meta-llama/llama-4-maverick",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Describe the image in detail. For context,the image is part of a research paper explaining the transformers architecture. Be specific about graphs, such as bar plots."},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{base64_image}"}
                    }
                ]
            }
        ]
    )

    # Récupérer la réponse
    description = completion.choices[0].message.content

    # Créer le chemin vers le fichier texte
    txt_path = os.path.splitext(image_path)[0] + ".txt"

    # Écrire la description dans le fichier texte
    with open(txt_path, "w", encoding="utf-8") as txt_file:
        txt_file.write(description)

    # Supprimer l'image originale
    os.remove(image_path)

    return "OK"


In [ ]:

def process_all_images_in_directory(root_dir: str):
    for dirpath, _, filenames in os.walk(root_dir):
        for filename in filenames:
            ext = os.path.splitext(filename)[1].lower()
            if ext in IMAGE_EXTENSIONS:
                image_path = os.path.join(dirpath, filename)
                try:
                    result = image_to_text(image_path)
                    print(f"{image_path} => {result}")
                except Exception as e:
                    print(f"Erreur lors du traitement de {image_path} : {e}")


In [ ]:
process_all_images_in_directory(marker_data_folder)

/content/drive/MyDrive/output_marker/26.258-i00/_page_58_Figure_1.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_0_Picture_1.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_8_Figure_5.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_10_Picture_5.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_10_Picture_7.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_22_Figure_0.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_25_Figure_1.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_28_Figure_1.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_32_Picture_0.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_33_Figure_0.jpeg => OK
/content/drive/MyDrive/output_marker/22847-i20/_page_34_Figure_1.jpeg => OK


# chunker

## requirments

In [8]:
# Install required libraries for Google Colab
!pip install --quiet langchain>=0.0.302 python-docx>=0.8.11 nltk>=3.8.1 tiktoken>=0.5.0


ERROR: Operation cancelled by user


## md fils to chunks txt

In [9]:
# Import section
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from docx import Document as DocxReader
from langchain.schema import Document
import nltk
from nltk.tokenize import sent_tokenize
import os

# Constants and configuration
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 200
DEFAULT_WINDOW_SIZE = 5
DEFAULT_STEP_SIZE = 2
SEPARATORS = ["\n\n", "\n"]

def download_nltk_dependencies():
    nltk.download('punkt_tab')

def load_documents(directory):
    documents = []
    for filename in os.listdir(directory):
        path = os.path.join(directory, filename)

        if filename.endswith(".docx"):
            docx = DocxReader(path)
            text = "\n".join([para.text for para in docx.paragraphs])
            documents.append(Document(page_content=text, metadata={"source": filename}))

        elif filename.endswith(".md"):
            with open(path, 'r', encoding='utf-8') as f:
                text = f.read()
                documents.append(Document(page_content=text, metadata={"source": filename}))

    return documents

def create_text_splitter():
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        disallowed_special=(),
        separators=SEPARATORS
    )
    return splitter

def apply_sentence_windows(doc, window_size, step_size):
    windows = []
    text = doc.page_content
    sentences = sent_tokenize(text)

    if len(sentences) <= window_size:
        new_doc = doc.copy()
        new_doc.page_content = " ".join(sentences)
        windows.append(new_doc)
    else:
        for i in range(0, len(sentences) - window_size + 1, step_size):
            window = sentences[i:i + window_size]
            new_doc = doc.copy()
            new_doc.page_content = " ".join(window)
            windows.append(new_doc)

    return windows

def save_documents_to_folder(documents, output_folder, prefix="doc", include_metadata=True):
    """
    Save processed documents to a folder for later use in embedding or other steps.

    Args:
        documents (list): List of Document objects (from LangChain) to save
        output_folder (str): Path to the folder where documents will be saved
        prefix (str): Prefix for the document filenames
        include_metadata (bool): Whether to include metadata in the saved files

    Returns:
        str: Path to the output folder
    """
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save each document
    saved_count = 0
    for i, doc in enumerate(documents):
        try:
            # Create a filename
            filename = f"{prefix}_{i+1}.txt"
            file_path = os.path.join(output_folder, filename)

            # Save the document content
            with open(file_path, 'w', encoding='utf-8') as f:
                # Ensure we have page_content
                if hasattr(doc, 'page_content'):
                    f.write(doc.page_content)
                else:
                    f.write(str(doc))

            saved_count += 1
        except Exception as e:
            print(f"Error saving document {i+1}: {str(e)}")

    print(f"Saved {saved_count} documents to {output_folder}")
    return output_folder, saved_count == len(documents)

def prepare_and_split_docs_by_sentence(directory, output_folder="processed_docs", window_size=DEFAULT_WINDOW_SIZE,
                                       step_size=DEFAULT_STEP_SIZE, prefix="window", save=True):
    """
    Process documents from a directory, split them into sentence-level chunks with sliding window,
    and optionally save them to disk.

    Args:
        directory (str): Path to the directory containing documents
        output_folder (str): Path where processed documents will be saved
        window_size (int): Number of sentences per window
        step_size (int): Number of sentences to slide the window for each chunk
        prefix (str): Prefix for saved document filenames
        save (bool): Whether to save the documents to disk

    Returns:
        tuple: (list of Document objects or output directory path, success status)
    """
    try:
        # Download NLTK dependencies
        download_nltk_dependencies()

        # Create and use document loaders
        documents = load_documents(directory)

        if not documents:
            print("No documents were found or loaded.")
            return None, False

        # Apply initial chunking
        pre_splitter = create_text_splitter()
        pre_split_docs = pre_splitter.split_documents(documents)

        # Apply sentence-level sliding window
        sentence_windows = []
        for doc in pre_split_docs:
            windows = apply_sentence_windows(doc, window_size, step_size)
            sentence_windows.extend(windows)

        print(f"Documents are split into {len(sentence_windows)} sentence windows")

        # Save documents if requested
        if save and sentence_windows:
            output_path, success = save_documents_to_folder(
                sentence_windows,
                output_folder,
                prefix=prefix
            )
            return output_path, success

        # Return the documents if not saving
        return sentence_windows

    except Exception as e:
        print(f"Error in document processing: {str(e)}")



ModuleNotFoundError: No module named 'docx'

In [ ]:
import os
#from chunk import prepare_and_split_docs_by_sentence

def main():
    # Debug: Print starting message
    print("Starting the document processing script...")

    base_dir = marker_data_folder

    # Define input and output paths
    data_dir = os.path.join(base_dir, "test")
    output_folder = os.path.join(base_dir, "processed_data")

    # Check if data directory exists
    if not os.path.exists(data_dir):
        print(f"Error: Data directory '{data_dir}' does not exist")

    # Debug: Print data directory path
    print(f"Data directory exists: {data_dir}")

    # Run the document processing
    print("Running the document processing function...")
    result, success = prepare_and_split_docs_by_sentence(
        directory=data_dir,
        output_folder=output_folder,
        window_size=5,
        step_size=2,
        prefix="chunk",
        save=True
    )

    # Report results
    if success:
        print("Document processing completed successfully!")
        print(f"Processed documents saved to: {result}")
    else:
        print("Document processing failed.")

if __name__ == "__main__":
    main()


In [ ]:

def main():
    print("Starting the document processing script...")

    base_dir = marker_data_folder

    # Loop over all subfolders in base_dir
    for folder_name in os.listdir(base_dir):
        data_dir = os.path.join(base_dir, folder_name)
        output_folder = os.path.join(base_dir, folder_name)
        result, success = prepare_and_split_docs_by_sentence(
                directory=data_dir,
                output_folder=output_folder,
                window_size=5,
                step_size=2,
                prefix="chunk",
                save=True
            )

            if success:
                print(f"Document processing completed successfully for '{folder_name}'Processed documents saved to: {result}")
            else:
                print(f"Document processing failed for '{folder_name}'.")



if __name__ == "__main__":
    main()
